In [1]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy
device = "cuda" if torch.cuda.is_available() else "cpu"
device

from time import sleep
from tqdm import tqdm

In [2]:
%reload_ext autoreload
%autoreload 2
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
import gym_super_mario_bros.actions as JoypadActions

from lib.env_wrappers import EnvWrapperFactory
from agents.ForgetfulAgent import ForgetfulAgent
from lib.MetricLogger import MetricLogger


In [3]:
imageShape = (84, 84)
actionShape = len(JoypadActions.SIMPLE_MOVEMENT)

In [4]:
%reload_ext autoreload
%autoreload 2
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, JoypadActions.SIMPLE_MOVEMENT)
env = EnvWrapperFactory.convert(env, shape=imageShape)
state = env.reset()
print(state.shape)

shape before any transformations: (240, 256, 3)
shape after grayscaler: (240, 256)
shape after resizer: (84, 84)
shape after all transformations: (5, 84, 84)
(5, 84, 84)


In [5]:
env.observation_space.shape

(5, 84, 84)

In [6]:
%reload_ext autoreload
%autoreload 2
agent = ForgetfulAgent(state_shape=env.observation_space.shape, action_shape=actionShape, device=device, net_name="CNN84x84")
save_dir = Path("logs") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
save_dir.mkdir(parents=True)
logger = MetricLogger(save_dir)
print(agent.name)

ForgetfulAgent-CNN84x84


In [ ]:
# try training

# maxStepsPerEpisode = 10_00_0000
learnCount = 0
episodes = 10000
epsRewards = []
with tqdm(range(1, episodes+1)) as tepoch:
    for eps in tepoch:
        tepoch.set_description(f"Episode {eps}")
        state = env.reset()
        epsReward = 0.0
        maxX = 0
        # print(f"starting episode: {eps}")
        # for i in range(maxStepsPerEpisode):
        while True:

            action = agent.getAction(state)
            next_state, reward, done, info = env.step(action)

            if info["flag_get"]:
                print(f"reached a flag")
                print(info)
                reward += 1000

            #episode stats
            epsReward += reward
            if info["x_pos"] > maxX:
                maxX = info["x_pos"]
                
            

            # add to memory
            agent.cache(state, next_state, action, reward, done)

            q, loss = agent.learn()

            logger.log_step(reward, loss, q)

            state = next_state

            # if q is not None:
            #     learnCount += 1
            #     if learnCount % 1000 == 0:
            #         print(f"q:{q}, loss={loss}")

            if info["flag_get"]:
                print(f"reached a flag")
                print(info)

            if done or info["flag_get"]:
                break

        # print(f"done: {done},\n info: {info}")
        logger.log_episode()
        if eps % 100 == 0:
            logger.record(episode=eps, epsilon=agent.exploration_rate, step=agent.current_step)
            # print(info)
            # agent.save(dir="models/", epoch=eps)
        if eps % 1000 == 0:
            # print(info)
            agent.save(dir="models/", epoch=eps)

        epsRewards.append(epsReward)
        tepoch.set_postfix_str(f"episode reward :{epsReward}, max X: {maxX}")
        # print("Total reward after episode {} is {}".format(eps, epsReward))
        

Episode 3:   0%|          | 2/10000 [00:05<8:00:24,  2.88s/it, episode reward :1190.0, max X: 867] 

In [20]:

env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, JoypadActions.SIMPLE_MOVEMENT)
env = EnvWrapperFactory.convert(env, shape=imageShape)


done = True
count = 0 
for step in range(10000):
    if done:
        count += 1
        if count > 2:
            break
        state = env.reset()
    # state, reward, done, info = env.step(agent.getBestAction(state))
    state, reward, done, info = env.step(agent.getAction(state))
    # state, reward, done, info = env.step(agent._exploit(state))
    env.render()
env.close()

shape before any transformations: (240, 256, 3)
shape after grayscaler: (240, 256)
shape after resizer: (84, 84)
shape after all transformations: (5, 84, 84)


In [11]:
agent.save(dir="models/", epoch=eps)

saving model to models/ForgetfulAgent-CNN84x84-checkpoint-9999.pytorch


In [15]:
print(agent.exploration_rate)

0.12048704660553211


In [ ]:

imageShape = (84, 84)
actionShape = len(JoypadActions.SIMPLE_MOVEMENT)

env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, JoypadActions.SIMPLE_MOVEMENT)
env = EnvWrapperFactory.convert(env, shape=imageShape)

%reload_ext autoreload
%autoreload 

agent = ForgetfulAgent(state_shape=env.observation_space.shape, action_shape=actionShape, device=device, net_name="CNN84x84")
agent.load("models/ForgetfulAgent-CNN84x84-checkpoint-Apr-08-2022-9000.pytorch")


done = True
count = 0 
for step in range(1000):
    if done:
        count += 1
        if count > 2:
            break
        state = env.reset()
    state, reward, done, info = env.step(agent.getBestAction(state))
    print(f"{info['x_pos']} {info['time']}")
    if done:
        print(info)
    env.render()
env.close()